<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>



# **Space X  Falcon 9 First Stage Landing Prediction**


# Exploratory Data Analysis with SQL



## Objectives
1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
%load_ext sql

In [3]:
import sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [2]:
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [5]:
df['Landing_Outcome'][5:20]

5       Uncontrolled (ocean)
6                 No attempt
7                 No attempt
8         Controlled (ocean)
9         Controlled (ocean)
10                No attempt
11                No attempt
12      Uncontrolled (ocean)
13      Failure (drone ship)
14        Controlled (ocean)
15                No attempt
16      Failure (drone ship)
17                No attempt
18    Precluded (drone ship)
19      Success (ground pad)
Name: Landing_Outcome, dtype: object

In [6]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

### 1. Display the names of the unique launch sites  in the space mission


In [8]:
%%sql
SELECT DISTINCT "Launch_Site" 
FROM SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### 2. Display 5 records where launch sites begin with the string 'CCA' 


In [9]:
%sql SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### 3. Display the total payload mass carried by boosters launched by NASA (CRS)


In [9]:
%%sql 
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass 
FROM SPACEXTABLE 
WHERE Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


Total_Payload_Mass
45596


### 4. Display average payload mass carried by booster version F9 v1.1


In [10]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS Average_Payload_Mass
FROM SPACEXTABLE 
WHERE Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


Average_Payload_Mass
2928.4


### 5. List the date when the first succesful landing outcome in ground pad was acheived.


In [11]:
%%sql
SELECT Date 
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (ground pad)'
ORDER BY Date ASC LIMIT 1

 * sqlite:///my_data1.db
Done.


Date
2015-12-22


### 6. List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [12]:
%%sql
SELECT DISTINCT Booster_Version FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (drone ship)' AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### 7. List the total number of successful and failure mission outcomes


In [13]:
%%sql
SELECT Mission_Outcome, COUNT(*) AS COUNT FROM SPACEXTABLE
GROUP BY Mission_Outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,COUNT
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### 8. List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [14]:
%%sql
SELECT Booster_Version FROM SPACEXTABLE 
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### 9. List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.



In [15]:
%%sql
SELECT 
    strftime('%m', Date) AS Month, 
    Booster_Version, 
    Launch_Site, 
    Landing_Outcome 
FROM 
    SPACEXTABLE 
WHERE 
    Landing_Outcome LIKE '%failure (drone ship)%' 
    AND strftime('%Y', Date) = '2015';

 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Launch_Site,Landing_Outcome
01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


### 10. Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [16]:
%%sql
SELECT 
    Landing_Outcome, 
    COUNT(*) AS Outcome_Count
FROM 
    SPACEXTABLE
WHERE 
    Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY 
    Landing_Outcome
ORDER BY 
    Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


---

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
